# Putting Redux

For this final assessment, you'll be asked to revisit the lag putting problem we did early on in the term.  But now we have the tools to create a much more realistic model: we can deal with the continuous changes in the slope of the green, we know how to deal with vectors, and we have a range of optimization tools at our disposal.

<br>

---

## The Green

The details for the green are included here.  As you will see, the green data is formatted in a way that will allow you to access the slope of the green at any given coordinate point.  To help you visualize the green, here are two plots: the first of the elevation on the green, and the second of the slope of the green (You can look at this code to see where this is coming from, but don't necessarily need to).

<br>

I've included a second "green" here with a very simple topography that you might use to test your simulation, since its much easier to guess how the ball should respond to this simpler surface.  To use the "test green", just comment out the first `filename` and uncomment the second one.

In [ ]:
#@title
import pandas as pd
import numpy as np
from scipy.interpolate import interpn
import matplotlib.pyplot as plt

# Upload elevation data for the green and put data in an array
# Switch to the second filename to do simple testing of your simulation
filename = 'https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Data/CT8_golf_green_data.xlsx'
#filename = 'https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Data/CT8_golf_green_data_test.xlsx'
data = pd.read_excel(filename, header=0, index_col=0)
green = np.array(data)

# Calculate the gradients at the green so that they can
# be accessed using coordinate values
grad = np.gradient(green)
grad[0] = np.flip(np.rot90(grad[0]),1)
grad[1] = np.flip(np.rot90(grad[1]),1)
# 'Gradient()' finds the gradient per data point; so 'grad'
# is divided by the distance between data points here to give
# gradient per meter
xstep = 1.525
grad = np.flip(grad)/xstep

# Plot a topographic map of the green
fig, (ax_topo, ax_slope) = plt.subplots(nrows=1, ncols=2, figsize=(14,10))
cs = ax_topo.contour(np.array(data.columns), np.array(data.index), green, levels=15)
ax_topo.clabel(cs,colors='black', fmt = '%1.3f');
ax_topo.set(title='Elevation Map of Green', xlabel='X-coordinate', ylabel='Y-coordinate');

# Plot the slope of the green (arrows pointing downhill)
x, y = np.meshgrid(np.array(data.index), np.array(data.columns))
ax_slope.quiver(y,x,-grad[0],-grad[1]);
ax_slope.set(title='Slope Map of Green', xlabel='X-coordinate');

Note that both the coordinate values and the elevation values are in meters.  

<br>

For now, change the "green" that you will be using to the test green (by commenting out and uncommenting the `filename` in the code above and rerunning the cell).

## Using the Green in your code

<br>

The code above produces the NumPy array `grad`, which holds the gradient values at a set of defined coordinate points, which are in regular grid with a spacing of 1.52 m.  For instance, we can access the slope of the green at the coordinate point (1.52 m,1.52 m) by doing the following:


In [ ]:
print(grad[0][1][1], grad[1][1][1])

Make sure you understand what these values mean by looking at the contour map above.  The units of the gradient are "meters per meter": how far vertically the surface changes for each meter of horizontal movement.  A positive value means the slope is going uphill as you move in the positive direction.
(Note that the map on the right shows the direction the ball would roll, so the signs are reversed on this plot).  

<br>

But `grad` by itself has a limited usefulness:

<br>

* Its arguments refer to index numbers, rather than distances
* More importantly, it can provide the slope only at a given set of data points, but you will need to access the slope at any plot on the green. So we need some interpolation, right?! We can find the slope at any point by interpolating between known points.

<br>

Since we need to interpolate in two dimensions, the tool we used earlier (`interp`) won't work.  Instead, we can use the SciPy function `interpn`, which interpolates an x- and y- component of the slope for any point on the green.  This code returns the x-component of the slope at the point $(1.5 ~m, 1.5 ~m)$.

In [ ]:
# Create an array that describes the location of known data points
points = (np.array(data.columns), np.array(data.index))

# Find the interpolated slope at the given point (here, (1.5, 1.5))
# in one direction (here 'grad[0]' indicates we want the x-directino)
# The 'bounds_error' tells the function to produce an error
# when the ball goes outside the known data points
slope_array = interpn(points,grad[0],[1.5,1.5], bounds_error=True)

# 'interpn' returns an array of length 1; for convenience, the next line takes the value
# out of the array and creates a simple float (decimal) number
interp_slope = slope_array[0]
interp_slope

`points` creates an array of the known points: you'll need to keep that in your `system` object.  `interpn` returns the x-component of the slope if `grad[0]` is entered as the second argument; if `grad[1]` is the second argument, the return value is the y-component of the slope.

### Step 1: Making a state object

For a test run, choose your initial starting point near the horizontal center of the test green but near the vertical bottom ($(6.0, 1.0)$).  Provide an initial velocity of $4~m/s$ in the positive y-direction.



In [ ]:
# Make a state object and assign initial values


### Step 2: Making a system object

Now define any other parameters you think you might need, and pack them into a `system` dictionary.  You'll need a coefficient of rolling friction ($\mu = 0.15$), as well as a mass ($0.046~kg$).  You can assume that the coefficient of drag is $C_d = 0.7$, the radius of the ball is $r = 2.16 cm$, and the density of air is $1.2 kg/m^3$.  You'll also need some simulation parameters, like the length of the simulation and time step.  Finally, you'll need to pack `points` and `grad` into the system to use later.

<br>

You can always come back to add, change, or remove these parameters as necessary.

In [ ]:
# Make a system object


### Step 3: Making a change function with friction

Start by defining a change function that will only take friction into account (so the ball will not curve, which is caused by gravity).  A simple model for frictional force is:

<br>

$$F_f = \mu m g$$

<br>

Create a change function in which the only acting force is friction.  Remember that this is a force *vector*: it will act in a direction.  Friction, like drag, is a reactive force: it always acts in the direction opposite that of velocity.



In [ ]:
# Define the friction_simple change function
def change_func_fric(t,state,system):

In [ ]:
# Test your code on your initial time step
change_func_fric(0,state,system)

You should find that your y-velocity is a little less than $4~m/s$ and your y-position is slightly higher than 1.0.

### Step 4: Running and Plotting the Simulation

Borrow the `run_simulation()` code from one of the baseball notebooks, make any necessary adjustments (they will be small if there are any), and run the simulation with a small simulation length (try 1-2 seconds).   Plot the y-velocity of the ball: it should move straight forward and then slow down.  

<br>

Then increase your simulation time to the point where the ball has stopped.  Something strange might happen to your velocity (and the simulation) as the velocity reaches 0.  How can you change `run_simulation()` to solve this problem?  In your fix, though, remember that negative velocities are not always a sign that something is wrong, though!  

<br>

(If you don't get strange results, you still want to fix the simulation to stop when the velocity reaches zero, just to save computational time).

In [ ]:
# Run the simulation

In [ ]:
# Plot y-velocity
results.vy.plot(title='Y-velocity of the putt')


Once you've gotten the simulation to stop when the ball does, go on to the next step.

### Step 5: Adding gravitational force

Gravitational force is also acting on the ball, pulling it down the hill.  Let's add that force now.  Write a new change function with both friction and gravity, which can be modeled with the following equation:

<br>

$$F_{grav} = mg \sin{\theta}$$

<br>

To calculate this, you'll need to use `interpn`, as explained at the beginning of the notebook.  You'll also need to convert a gradient (units: meters per meter) into an angle (units: radians).  This image of gradient of 0.1 might help you think about this:

<br>

<center>
<img src = https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Images/3_7/gradient_to_radians.PNG width = 500>
</center>

<br>

Notice that this force, too, will act in the x- and y-direction, so you'll need to find the gradient in both directions.

In [ ]:
# Define the change_func with gravity
# Start by copying and pasteing what you have above, but
# leave that working code (change_func_fric_simple) there
def change_func_grav(t,state,system):

In [ ]:
# Test your code on your initial time step
# You should see that there is now a negative x-velocity
# (which might be quite small)
change_func_grav(0,state,system)

In [ ]:
# Now run your simulation and plot the trajectory
results = run_simulation(system, state, change_func)
x = results.x.values
y = results.y.values
x_vs_y = pd.Series(data=y,index=x)
x_vs_y.plot(xlabel='x position (m)',ylabel='y position (m)',
            figsize=[5,8],title = 'Trajectory of the putt');

Is your ball curving down the hill and then stopping?  You're ready for the next step!

### Step 6: Adding drag

Drag force will not make as large a difference as gravity or friction, but its effects are not insignificant.   Write and test a change function that includes drag in the calculation.  This will look a lot like the drag function in the baseball notebooks!  

<br>  

You will probably find it easier to define a separate drag force function and call it in the change function, as we did in the baseball notebook (particularly since the drag function already exists in the earlier notebook).

<br>

When you have it working, check to see how much drag affects the putt by comparing the final y-position of the putt with the final y-position of the putt in step 5.



In [ ]:
# Put your code here

### Step 7: Optimizing the Putt

Alright, now we're going to use our simulation to lower our golf scores.  Your ball is sitting on the green (not the evenly sloped test green but the original one) at the location (2.0, 2.0).   The hole is located at (8.0, 18.0).  You want to have the ball stop as close to the hole as possible.

<br>

First, go up to the first couple cells of the notebook, and replace the test green with the real contoured green.

<br>

Next, experiment a little to figure out approximately how hard you need to hit it to get it to stop there.  You'll want to adjust your initial state values for velocity, too, to get a sense of which direction you need to hit the ball.  Use `plot_trajectory_contour` (below) so you can see how close your ball is to the hole.

<br>

Once you've decided on a pretty good velocity, find the best angle (measured from the positive x-axis) to hit the ball at that velocity by creating a `range_func` and sweeping through potential angles, much like we did in Notebook 3.7.2.  You'll need to adapt that `range_func`, and will need a couple more parameters in your system.

In [ ]:
# Put your code here

I've also include a fancier `plot_trajectory` that overlays the path of the ball onto the contour map: to use this function, you will need to call the coordinates of your hole location 'xh' and 'yh' (or you can change this function to match your variable names).

In [ ]:
def plot_trajectory_contour(results,system):
    x = results.x.values
    y = results.y.values
    x_vs_y = pd.Series(data=y,index=x)
    y_arr = ([system['yh']])
    x_arr = ([system['xh']])
    hole_loc = pd.Series(data=y_arr,index=x_arr)

    fig, ax = plt.subplots(figsize=(6,10))
    cs = ax.contour(np.array(data.columns), np.array(data.index), green, levels=15)
    ax.clabel(cs,colors='black', fmt = '%1.3f');
    ax.set(title='Optimized Lag Putt', xlabel='X-coordinate', ylabel='Y-coordinate');

    x_vs_y.plot(xlabel='x position (m)',
             ylabel='y position (m)',figsize=[6,9], xlim=[0,12.2],ylim=[0,21.3])
    hole_loc.plot(style = 'o')